# How to compute a correspondence with functional maps

This is the first demo on how to use geomfum to compute a correspondence with functional maps.  
In this notebook we will see what does it mean to compute a functional map and what are the main steps to do so.  
Lets Start!

# Importing and visualizing shapes

First of all, we need to import our data, we will start from a pair of shapes from the TOSCA dataset (https://paperswithcode.com/dataset/tosca)

In [24]:
from geomfum.dataset import NotebooksDataset
from geomfum.plot import MeshPlotter
from geomfum.shape import TriangleMesh

dataset = NotebooksDataset()

mesh_a = TriangleMesh.from_file(dataset.get_filename("cat-00"))
mesh_b = TriangleMesh.from_file(dataset.get_filename("lion-00"))


We can visualize our shapes with the MeshPlotter

In [25]:
# We selct plotting options one for all
PLOT_TYPE = "plotly"

if PLOT_TYPE == "pyvista":
    STATIC_VIZ = True
    import pyvista as pv

    if STATIC_VIZ:
        pv.set_jupyter_backend("static") 

In [ ]:
plotter_a = MeshPlotter.from_registry(which=PLOT_TYPE)
plotter_a.add_mesh(mesh_a)
plotter_a.show()

plotter_b = MeshPlotter.from_registry(which=PLOT_TYPE)
plotter_b.add_mesh(mesh_b)
plotter_b.show()

# Compute Basis
Now that we have our pair of shapes, we can compute their basis.  
The basis of a shape is a set of functions that represent a basis for the space of squared integrable function defined on the surface of the shape.  
There are different kind of basis, but usually we consider the eigenfunctions of the Laplace beltrami operator.

In [ ]:
k=50 #we select the number of basis that we want to compute
mesh_a.laplacian.find_spectrum(spectrum_size=k, set_as_basis=True)
mesh_b.laplacian.find_spectrum(spectrum_size=k, set_as_basis=True)


Here we display two eigenfunctions on the shape.

In [ ]:
plotter_a.colormap = plotter_b.colormap = "RdBu"
plotter_a.set_vertex_scalars(mesh_a.basis.vecs[:,10])
plotter_a.show()

plotter_b.set_vertex_scalars(mesh_b.basis.vecs[:,4])
plotter_b.show()

# Compute Descriptors
Another set of functions that are useful in functional maps are the descriptors.  
While the basis describes the function space, the descriptors are functions that describe the shapes, including geometric and semantic inforations that are shared by the shapes. 

In [29]:
import numpy as np

from geomfum.descriptor.pipeline import (
    ArangeSubsampler,
    DescriptorPipeline,
    L2InnerNormalizer,
)
from geomfum.descriptor.spectral import HeatKernelSignature

**Landmarks**  
Often, the information provided by descriptors alone is not sufficient to compute an accurate functional map.  
For this reason, a good thing is to provide some points that we know are in correspondence, these are called landmarks.
We can select the landmarks plotting the shapes.

In [30]:
mesh_a.landmark_indices =np.array( [2840, 1594,5596, 6809,3924,7169] )
mesh_b.landmark_indices =np.array( [1334,834,4136,4582,3666,4955])

In [31]:
steps = [
    HeatKernelSignature.from_registry(n_domain=100, use_landmarks=True),
    ArangeSubsampler(subsample_step=1),
    L2InnerNormalizer(),
]

pipeline = DescriptorPipeline(steps)

descr_a = pipeline.apply(mesh_a)
descr_b = pipeline.apply(mesh_b)

In [ ]:
plotter_a.colormap = plotter_b.colormap = "viridis"
plotter_a.set_vertex_scalars(descr_a[0])
plotter_a.show()
plotter_b.set_vertex_scalars(descr_b[0])
plotter_b.show()

# Optimize Functional Map
Now, we have all the elements to optimize our first functional maps, we can select different energies and weight them differently.

In [33]:
from geomfum.functional_map import (
    FactorSum,
    LBCommutativityEnforcing,
    OperatorCommutativityEnforcing,
    SpectralDescriptorPreservation,
)
from geomfum.numerics.optimization import ScipyMinimize


We select the numbr of eigenfunctions we want to use.

In [34]:
mesh_a.basis.use_k=20
mesh_b.basis.use_k=20

In [35]:
factors = [
    SpectralDescriptorPreservation(
        mesh_a.basis.project(descr_a),
        mesh_b.basis.project(descr_b),
        weight=1.0,
    ),
    LBCommutativityEnforcing.from_bases(
        mesh_a.basis,
        mesh_b.basis,
        weight=1e-2,
    ),
    OperatorCommutativityEnforcing.from_multiplication(
        mesh_a.basis, descr_a, mesh_b.basis, descr_b, weight=0
    ),
    OperatorCommutativityEnforcing.from_orientation(
        mesh_a, descr_a, mesh_b, descr_b, weight=1e-1
    ),
]

objective = FactorSum(factors)

optimizer = ScipyMinimize(
    method="L-BFGS-B",
)

x0 = np.zeros((mesh_b.basis.spectrum_size, mesh_a.basis.spectrum_size))

res = optimizer.minimize(
    objective,
    x0,
    fun_jac=objective.gradient,
)

fmap = res.x.reshape(x0.shape)


We show the obtained functional maps.

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(fmap, cmap='bwr')

# Get the correspondence
Once we have the functional map, we can compute the point-to-point correspondence performing a nearest search in the space of function.

In [37]:
from geomfum.convert import P2pFromFmConverter

p2p_converter = P2pFromFmConverter()

p2p = p2p_converter(fmap, mesh_a.basis, mesh_b.basis)

We can visualize the quality of the mapping on the shapes by transfering a vertex scalar.

In [ ]:
scalar_a=np.mean(mesh_a.vertices,axis=-1)

plotter_a.set_vertex_scalars(scalar_a)
plotter_a.show()

plotter_b.add_mesh(mesh_b)
plotter_b.set_vertex_scalars(scalar_a[p2p])
plotter_b.show()


# Refining the correspondence
A lot of time, a small number of basis is not enough to have a good correspondence, for this reason a lot of methods are based on a 'refinement' stage.

In [39]:
from geomfum.refine import ZoomOut

In [43]:
zoomout = ZoomOut(nit=6, step=5)

zoomout_fmap_matrix_ = zoomout(fmap, mesh_a.basis, mesh_b.basis)
p2p_ref=p2p_converter(zoomout_fmap_matrix_, mesh_a.basis, mesh_b.basis)

In [ ]:
plt.imshow(zoomout_fmap_matrix_, cmap='bwr')

In [ ]:
scalar_a=np.mean(mesh_a.vertices,axis=-1)

plotter_a.set_vertex_scalars(scalar_a)
plotter_a.show()

plotter_b.add_mesh(mesh_b)
plotter_b.set_vertex_scalars(scalar_a[p2p_ref])
plotter_b.show()
